In [1]:
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format



In [2]:
import configparser

config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','AWS_SECRET_ACCESS_KEY')



In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()


In [4]:

df = spark.read.json("s3a://udacity-dend/song_data/A/B/C/*.json")

In [5]:
dflog = spark.read.json("s3a://udacity-dend/log_data/2018/11/*.json")

In [12]:
df.show()

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARLTWXK1187FB5A3F8|       32.74863|      Fort Worth, TX|       -97.32925|         King Curtis|326.00771|        1|SODREIN12A58A7F2E5|A Whiter Shade Of...|   0|
|ARIOZCU1187FB3A3DC|           null|          Hamlet, NC|            null|       JOHN COLTRANE|220.44689|        1|SOCEMJV12A6D4F7667|Giant Steps (Alte...|   0|
|ARPFHN61187FB575F6|       41.88415|         Chicago, IL|       -87.63241|         Lupe Fiasco|279.97995|        1|SOWQTQZ12A58A7B63E|Streets On Fire (...|   0|
|AR5S9OB1187B9931E3|       34.0534

In [13]:
df.count()

23

In [6]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [14]:
dflog.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [15]:
dflog.count()

8056

In [6]:
df.createOrReplaceTempView("song_view")

In [7]:
song = spark.sql("SELECT song_id, title, artist_id, year, duration FROM song_view")

In [103]:
artist = spark.sql("SELECT artist FROM log_view")

In [104]:
artist.show()

+--------------------+
|              artist|
+--------------------+
|            Harmonia|
|         The Prodigy|
|               Train|
|         Sony Wonder|
|           Van Halen|
|           Magic Sam|
|Edward Sharpe & T...|
|Usher featuring w...|
|         Helen Reddy|
|        Taylor Swift|
|           Sean Paul|
|         Soundgarden|
|         The Killers|
|       Amy Winehouse|
|      Steve Anderson|
|          Rob Zombie|
|  Deadmau5 & Kaskade|
|        Shania Twain|
|      Los Campesinos|
|            Ill Nino|
+--------------------+
only showing top 20 rows



In [106]:
artist1 = spark.sql("SELECT artist_name FROM song_view")

In [108]:
artist1.show()

+--------------------+
|         artist_name|
+--------------------+
|         King Curtis|
|       JOHN COLTRANE|
|         Lupe Fiasco|
|         Bullet Boys|
|    The Bear Quartet|
|              Faunts|
|The Band of HM Ro...|
|    Styles Of Beyond|
|        Nasty Savage|
| Vince Guaraldi Trio|
|        Danilo Perez|
|     Brigitte Bardot|
|                Eels|
|         Steve Morse|
|        Eddie Sierra|
|  Theory In Practice|
|         Line Renaud|
|             Skinlab|
|      Fort Knox Five|
|        Gwen Stefani|
+--------------------+
only showing top 20 rows



In [8]:
song.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SODREIN12A58A7F2E5|A Whiter Shade Of...|ARLTWXK1187FB5A3F8|   0|326.00771|
|SOCEMJV12A6D4F7667|Giant Steps (Alte...|ARIOZCU1187FB3A3DC|   0|220.44689|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|ARPFHN61187FB575F6|   0|279.97995|
|SOMAPYF12A6D4FEC3E|All Day & All Of ...|AR5S9OB1187B9931E3|   0|156.62975|
|SOAPVNX12AB0187625|I Remember Nights...|AR5T40Y1187B9996C6|1998| 249.3122|
|SOFIUVJ12A8C13C296|Will You Tell Me ...|AR9OEB71187B9A97C6|2005|397.16526|
|SOBHXUU12A6D4F5F14|National Emblem (...|ARBDJHO1252CCFA6FC|   0|188.73424|
|SOQFYBD12AB0182188|               Intro|ARAADXM1187FB3ECDB|1999| 67.63057|
|SOYLILV12A8C136650|                 XXX|ARZJDBC1187FB52056|1984|327.00036|
|SOHTCZS12A6D4FC402|  The Christmas Song|AROSPS51187B9B481F|1965|197.95546|
|SONSKXP12A8

In [23]:
song.write.mode("overwrite").partitionBy("year","artist_id")

In [24]:
artist_table = spark.sql("SELECT artist_id, artist_location, artist_latitude, artist_longitude FROM song_view")

In [25]:
artist_table.show()

+------------------+--------------------+---------------+----------------+
|         artist_id|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+---------------+----------------+
|ARLTWXK1187FB5A3F8|      Fort Worth, TX|       32.74863|       -97.32925|
|ARIOZCU1187FB3A3DC|          Hamlet, NC|           null|            null|
|ARPFHN61187FB575F6|         Chicago, IL|       41.88415|       -87.63241|
|AR5S9OB1187B9931E3|     Los Angeles, CA|       34.05349|      -118.24532|
|AR5T40Y1187B9996C6|       Lulea, Sweden|           null|            null|
|AR9OEB71187B9A97C6|Edmonton, Alberta...|           null|            null|
|ARBDJHO1252CCFA6FC|                    |           null|            null|
|ARAADXM1187FB3ECDB|  Woodland Hills, CA|        34.1688|      -118.61092|
|ARZJDBC1187FB52056|    Brandon, Florida|       27.94017|       -82.32547|
|AROSPS51187B9B481F|                    |           null|            null|
|AR0IAWL1187B9A96D0|     

In [26]:
dflog.show()

+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|      auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|            Harmonia| Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|         The Prodigy| Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Su

In [29]:
dffilter = dflog.filter(dflog.page == 'NextSong')

TypeError: 'Column' object is not callable

In [30]:
dffilter.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|         The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyva

In [31]:
dffilter.createOrReplaceTempView("log_view")

In [33]:
dffilter.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [35]:
user = spark.sql('SELECT userId, firstName, lastName, gender, level FROM log_view')

In [36]:
user.show()

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    61|   Samuel|Gonzalez|     M| free|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    49|    Chloe|  Cuevas|     F| paid|
+------+---------+--------+------+-----+
only showing top

In [38]:
time = spark.sql("SELECT ts FROM log_view")

In [39]:
time.show()

+-------------+
|           ts|
+-------------+
|1542241826796|
|1542242481796|
|1542242741796|
|1542253449796|
|1542260935796|
|1542261224796|
|1542261356796|
|1542261662796|
|1542262057796|
|1542262233796|
|1542262434796|
|1542262456796|
|1542262679796|
|1542262728796|
|1542262893796|
|1542263158796|
|1542263378796|
|1542265716796|
|1542265929796|
|1542266927796|
+-------------+
only showing top 20 rows



In [74]:
@udf
def timeConv(ms):
    
    import datetime
    
    sec = int(ms)/1000
    
    result = datetime.datetime.fromtimestamp(sec).strftime('%Y-%m-%d %H:%M:%S.%f')

    return result

In [75]:
a = time.withColumn('datetime', timeConv("ts"))

In [76]:
a.show()

+-------------+--------------------+
|           ts|            datetime|
+-------------+--------------------+
|1542241826796|2018-11-15 00:30:...|
|1542242481796|2018-11-15 00:41:...|
|1542242741796|2018-11-15 00:45:...|
|1542253449796|2018-11-15 03:44:...|
|1542260935796|2018-11-15 05:48:...|
|1542261224796|2018-11-15 05:53:...|
|1542261356796|2018-11-15 05:55:...|
|1542261662796|2018-11-15 06:01:...|
|1542262057796|2018-11-15 06:07:...|
|1542262233796|2018-11-15 06:10:...|
|1542262434796|2018-11-15 06:13:...|
|1542262456796|2018-11-15 06:14:...|
|1542262679796|2018-11-15 06:17:...|
|1542262728796|2018-11-15 06:18:...|
|1542262893796|2018-11-15 06:21:...|
|1542263158796|2018-11-15 06:25:...|
|1542263378796|2018-11-15 06:29:...|
|1542265716796|2018-11-15 07:08:...|
|1542265929796|2018-11-15 07:12:...|
|1542266927796|2018-11-15 07:28:...|
+-------------+--------------------+
only showing top 20 rows



In [80]:
a.createOrReplaceTempView("time")

In [90]:
time_table = spark.sql("SELECT distinct(ts), datetime FROM time")

In [91]:
time_table.show()

+-------------+--------------------+
|           ts|            datetime|
+-------------+--------------------+
|1542295580796|2018-11-15 15:26:...|
|1542296032796|2018-11-15 15:33:...|
|1542299847796|2018-11-15 16:37:...|
|1542314793796|2018-11-15 20:46:...|
|1542779804796|2018-11-21 05:56:...|
|1542793114796|2018-11-21 09:38:...|
|1542842644796|2018-11-21 23:24:...|
|1542186737796|2018-11-14 09:12:...|
|1542195958796|2018-11-14 11:45:...|
|1543397770796|2018-11-28 09:36:...|
|1543437391796|2018-11-28 20:36:...|
|1541385481796|2018-11-05 02:38:...|
|1541424913796|2018-11-05 13:35:...|
|1541429962796|2018-11-05 14:59:...|
|1541435666796|2018-11-05 16:34:...|
|1542134866796|2018-11-13 18:47:...|
|1543603205796|2018-11-30 18:40:...|
|1542393418796|2018-11-16 18:36:...|
|1542675563796|2018-11-20 00:59:...|
|1542725885796|2018-11-20 14:58:...|
+-------------+--------------------+
only showing top 20 rows



In [77]:
dflog.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [79]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [114]:
songplays = spark.sql("""SELECT datetime, userId, level, song_id, artist_id, sessionId, location, userAgent
                      FROM time
                      JOIN log_view
                      ON time.ts == log_view.ts
                      JOIN song_view
                      ON song_view.artist_name == log_view.artist
                      
                      """)

In [115]:
songplays.show()

+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|            datetime|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|
+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|2018-11-15 13:20:...|    30| paid|SOTDCIR12AB0184574|ARZGTK71187B9AC7F5|      324|San Jose-Sunnyval...|Mozilla/5.0 (Wind...|
|2018-11-14 06:49:...|    16| paid|SODWBIK12AB017F87D|ARSMG8X1187B99CA99|      479|Birmingham-Hoover...|"Mozilla/5.0 (Mac...|
|2018-11-14 11:33:...|    15| paid|SOTDCIR12AB0184574|ARZGTK71187B9AC7F5|      557|Chicago-Napervill...|"Mozilla/5.0 (X11...|
|2018-11-14 13:11:...|    34| free|SOWQTQZ12A58A7B63E|ARPFHN61187FB575F6|      495|Milwaukee-Waukesh...|Mozilla/5.0 (Maci...|
|2018-11-14 20:16:...|   101| free|SORRZGD12A6310DBC3|ARVBRGZ1187FB4675A|      603|New Orleans-Metai...|"Mozilla/5.0 (